In [6]:
from panovlm.model import  PanoramaVLM
from train import VLMModule, safe_load_checkpoint
from pathlib import Path
import torch
import logging
from typing import Optional

logger = logging.getLogger(__name__)

def load_model_and_lora(checkpoint_path: str, lora_weights_path: Optional[str], device: torch.device, **model_kwargs) -> VLMModule:
    """
    1단계: 체크포인트와 LoRA 가중치를 로드하여 생성용 모델 준비
    """
    logger.info("=" * 60)
    logger.info("🚀 1단계: 모델 및 LoRA 가중치 로드")
    logger.info("=" * 60)
    
    # 체크포인트 로드
    logger.info(f"📂 체크포인트 로드: {checkpoint_path}")
    checkpoint = safe_load_checkpoint(checkpoint_path)
    if not checkpoint:
        raise ValueError(f"체크포인트 로드 실패: {checkpoint_path}")
    
    # LoRA 경로 자동 감지
    if lora_weights_path is None:
        checkpoint_dir = Path(checkpoint_path).parent
        potential_lora_path = checkpoint_dir / "lora_weights"
        if potential_lora_path.exists():
            lora_weights_path = str(potential_lora_path)
            logger.info(f"🔍 LoRA 가중치 자동 감지: {lora_weights_path}")
    
    # 모델 로드 (finetune 단계)
    model = VLMModule.load_from_checkpoint(
        checkpoint_path,
        stage="finetune",
        map_location=device,
        strict=False,
        **model_kwargs
    )
    
    # LoRA 가중치 로드
    if lora_weights_path and Path(lora_weights_path).exists():
        logger.info(f"🔧 LoRA 가중치 로드: {lora_weights_path}")
        
        # LoRA 파일 구조 검증
        lora_path = Path(lora_weights_path)
        adapter_config = lora_path / "adapter_config.json"
        adapter_model = lora_path / "adapter_model.safetensors"
        
        if adapter_config.exists() and adapter_model.exists():
            success = model.model.load_lora_weights(lora_weights_path)
            if success:
                logger.info("✅ LoRA 가중치 로드 성공!")
                
                # LoRA 설정 정보 출력
                lora_info = model.model.get_lora_info()
                if lora_info.get("is_lora_enabled", False):
                    logger.info(f"📊 LoRA 설정 - Rank: {lora_info.get('lora_r')}, Alpha: {lora_info.get('lora_alpha')}")
                    logger.info(f"   Target modules: {lora_info.get('target_modules')}")
            else:
                logger.warning("⚠️ LoRA 가중치 로드 실패, 기본 모델로 진행")
        else:
            logger.warning(f"⚠️ LoRA 파일 누락: {lora_weights_path}")
    else:
        logger.info("📝 LoRA 가중치 없음, 기본 모델 사용")
    
    # 평가 모드 설정
    model.eval()
    model = model.to(device)
    model.model.requires_grad_(False)
    
    logger.info(f"✓ 모델 준비 완료 - Device: {device}, Stage: {model._stage_key}")
    return model

model = load_model_and_lora(
    checkpoint_path="runs/siglipv2qwen25Instruct_e2p_finetune_mlp/best.ckpt",
    lora_weights_path="runs/siglipv2qwen25Instruct_e2p_finetune_mlp/lora_weights",  # LoRA 가중치 경로를 지정하거나
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)

2025-08-18 10:12:50,405 - __main__ - INFO - ============================================================
2025-08-18 10:12:50,407 - __main__ - INFO - 🚀 1단계: 모델 및 LoRA 가중치 로드
2025-08-18 10:12:50,408 - __main__ - INFO - ============================================================
2025-08-18 10:12:50,410 - __main__ - INFO - 📂 체크포인트 로드: runs/siglipv2qwen25Instruct_e2p_finetune_mlp/best.ckpt
2025-08-18 10:12:51,630 - panovlm.utils - INFO - Successfully loaded checkpoint: runs/siglipv2qwen25Instruct_e2p_finetune_mlp/best.ckpt
2025-08-18 10:12:53,103 - train - INFO - VICReg loss weight set to: 0.0 for stage: finetune
[Tokenizer Setup] Warning: No bos_token defined
[Tokenizer Setup] Warning: No unk_token defined
[Tokenizer Setup] Added 1 special tokens: ['<|vision|>', '<|endoftext|>']
[Tokenizer Setup] Resized embeddings: 151936 -> 151666
[Tokenizer Setup] Final token configuration:
  - Vocabulary size: 151666 (was 151665)
  - pad_token: '<|endoftext|>' (id: 151643)
  - eos_token: '<|im_end|>' 

In [ ]:
from panovlm.dataset import ChatPanoDataset